# Installation and setup

In [1]:
!pip install cdqa

     |████████████████████████████████| 51kB 9.1MB/s 
     |████████████████████████████████| 286kB 51.4MB/s 
     |████████████████████████████████| 10.5MB 34.8MB/s 
     |████████████████████████████████| 317kB 53.4MB/s 
     |████████████████████████████████| 6.7MB 22.5MB/s 
     |████████████████████████████████| 748.9MB 17kB/s 
     |████████████████████████████████| 92kB 13.4MB/s 
     |████████████████████████████████| 51kB 9.3MB/s 
     |████████████████████████████████| 870kB 49.2MB/s 
     |████████████████████████████████| 1.0MB 48.5MB/s 
  Created wheel for cdqa: filename=cdqa-1.3.9-cp36-none-any.whl size=47640 sha256=1bb9fd429be7821d916cc28459d9b4c4d5ca523c6602a4271658aa7dea07f7bf
  Stored in directory: /root/.cache/pip/wheels/8b/9a/68/d3f7651ea29c30d1bebc9e946bf5a8cf922e1c86fb6b8a33d9
  Created wheel for tika: filename=tika-1.19-cp36-none-any.whl size=29222 sha256=801ab8785e05dd882608df502f06b63fe1120828396eba7107d3606326aafc13
  Stored in directory: /root/.cache/pip/whee

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


# Baseline

In [0]:
from cdqa.utils.download import download_squad, download_model

directory = 'models'

# Downloading data
# download_squad(dir=directory)

# Downloading pre-trained BERT fine-tuned on SQuAD 1.1
download_model('bert-squad_1.1', dir=directory)


bert_qa.joblib already downloaded


In [0]:
import pandas as pd
from ast import literal_eval
from cdqa.pipeline import QAPipeline

df = pd.read_csv('titleparagraph.csv', converters={'Paragraphs': literal_eval},nrows=1000)
df.to_csv('titleparagraph_1000.csv')


In [0]:
# df=df.drop(['Unnamed: 0'],axis=1)
# df=df.rename(columns={'Paragraphs':'paragraphs'})

In [0]:
len(df['title'])

106925

In [0]:
#df.to_csv('titleparagraph.csv',index=False)

In [0]:
df2=df[:21385]
df2['title']
df2.head(100)

,title,paragraphs
0,0,"[""Email marketing - Wikipedia Email marketing..."
1,1,"[""The Mother ( How I Met Your Mother ) - wikip..."
2,3,"[""List of National Football League career quar..."
3,6,"[""Pom Klementieff - wikipedia Pom Klementieff..."
4,15,"[""Nashville sound - wikipedia Nashville sound..."
5,19,"[""List of Premier League broadcasters - wikipe..."
6,21,"[""Learner 's permit - wikipedia Learner 's pe..."
7,22,"[""God 's Not Dead : a Light in Darkness - Wiki..."
8,27,"[""President of the United Nations General Asse..."
9,30,"[""Powerball - wikipedia Powerball Jump to : ..."


In [0]:
cdqa_pipeline = QAPipeline(reader='models/bert_qa.joblib') # use 'distilbert_qa.joblib' for DistilBERT instead of BERT
cdqa_pipeline.fit_retriever(df=df2)

ValueError: zero-dimensional arrays cannot be concatenated

In [0]:
query = 'which is the most common use of opt-in e-mail marketing'
prediction = cdqa_pipeline.predict(query=query)
print('query: {}'.format(query))
print('answer: {}'.format(prediction[0]))
print('title: {}'.format(prediction[1]))
print('paragraph: {}'.format(prediction[2]))

query: which is the most common use of opt-in e-mail marketing
answer: business email addresses
title: 0
paragraph: Email marketing - Wikipedia  Email marketing  Jump to : navigation , search      ( hide ) This article has multiple issues . Please help improve it or discuss these issues on the talk page . ( Learn how and when to remove these template messages )      This article needs additional citations for verification . Please help improve this article by adding citations to reliable sources . Unsourced material may be challenged and removed . ( September 2014 ) ( Learn how and when to remove this template message )         This article possibly contains original research . Please improve it by verifying the claims made and adding inline citations . Statements consisting only of original research should be removed . ( January 2015 ) ( Learn how and when to remove this template message )    ( Learn how and when to remove this template message )       Part of a series on     Internet

# Create small sample of test json file

In [0]:
import json
with open('final_dictshort_test.json', 'r') as f:
    train_data = json.load(f)

In [0]:
test_data=train_data['data'][:3]
test_data={'data':test_data}


with open('final_dictshort_test_3points.json', 'w') as fp:
    json.dump(test_data, fp)

# Evaluation from Cdqa

In [0]:
from cdqa.utils.evaluation import evaluate_pipeline

evaluate_pipeline(cdqa_pipeline, 'final_dictshort_test_3points.json')


Evaluation results: {'exact_match': 0.0, 'f1': 0.0}


{'exact_match': 0.0, 'f1': 0.0}

# Jacardian similarity evaluation

In [0]:
def get_jaccard_sim(str1, str2): 
    a = set(str1.split()) 
    b = set(str2.split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

In [0]:
score=get_jaccard_sim('business email addresses',"a newsletter sent to an advertising firm 's customers")

In [0]:
score

0.0

In [0]:
dd=pd.read_csv('short_answers.csv',nrows=10)

In [0]:
dd['short_answers'][0]

"a newsletter sent to an advertising firm 's customers"